Load the Libraries and Data

In [ ]:
#Load the libraries
import os
import numpy as np 
from nibabel.testing import data_path
import nibabel as nib
import matplotlib.pyplot as plt
import pandas as pd
from nilearn import image


In [ ]:
#Path to the data

datapath='/data/neuromark2/Data/ABCD/Data_BIDS_5/Raw_Data/'

SubIDs=os.listdir(datapath)

timepoint="/Baseline/"
len(SubIDs)

#Get the foldernames from the SubIDs
dict1={}

for s in SubIDs:
    try:
        folder=os.listdir(datapath+s+timepoint)
        for f in folder:
            if 'anat' in f and 'NORM' not in f:
                dict1[s]=[f]
                break
            if 'anat' in f:
                dict1[s]=[f]
    except:
        print("Path not found",s)
        pass
print(len(dict1))

In [ ]:
# QC data
df = pd.read_csv("/data/neuromark2/Data/ABCD/Data_info/Demo51/abcd-data-release-5.1/core/imaging/mri_y_qc_incl.csv")  
df1=df[df['eventname']=='baseline_year_1_arm_1']
qc_df=df1[['src_subject_id','imgincl_t1w_include']]
print(qc_df.shape)

#CBCL data
df = pd.read_csv("/data/neuromark2/Data/ABCD/Data_info/Demo51/abcd-data-release-5.1/core/mental-health/mh_p_cbcl.csv")  
df1=df[df['eventname']=='baseline_year_1_arm_1']
cbcl_df=df1[['src_subject_id','cbcl_scr_syn_anxdep_r']]
print(cbcl_df.shape)

#Gender data
df = pd.read_csv("/data/neuromark2/Data/ABCD/Data_info/Demo51/abcd-data-release-5.1/core/abcd-general/abcd_p_demo.csv")
df1=df[df['eventname']=='baseline_year_1_arm_1']
gender_df=df1[['src_subject_id','demo_sex_v2']]
print(gender_df.shape)

#Site data
df = pd.read_csv("/data/neuromark2/Data/ABCD/Data_info/Demo51/abcd-data-release-5.1/core/abcd-general/abcd_y_lt.csv")
df1=df[df['eventname']=='baseline_year_1_arm_1']
site_df=df1[['src_subject_id','site_id_l','interview_age]]
print(site_df.shape)

#Merge these based on the subject ids
merged_df1=pd.merge(qc_df,cbcl_df,on='src_subject_id',how='inner')
merged_df2=pd.merge(gender_df,site_df,on='src_subject_id',how='inner')
merged_df=pd.merge(merged_df1,merged_df2,on='src_subject_id',how='inner')
merged_df['src_subject_id']=merged_df['src_subject_id'].str.replace('_','')


In [ ]:
dict_df = pd.DataFrame.from_dict(dict1, orient='index',columns=['scan_session'])

dict_df['src_subject_id'] = dict_df.index

dict_df = dict_df.reset_index(drop=True)

final_df = pd.merge(merged_df, dict_df,on='src_subject_id',how='inner')

filename='Data.csv'
final_df.to_csv(filename)

Create mean image and mask from first 1000 images

In [ ]:
df = pd.read_csv("Data.csv")  
df=df[df['imgincl_t1w_include']==1]
df=df[:1000]
df.shape

In [ ]:
# Create (121,145,121,subjects) image
images=[]
for _,s in df.iterrows():
    d=s['src_subject_id']
    t=s['scan_session']
    path=datapath+d+timepoint+t
    file_path=os.path.join(path,'smwc1pT1.nii')
    images.append(nib.load(file_path).get_fdata()) 
stacked_image = np.stack(images, axis=-1) 

print(stacked_image.shape)

In [ ]:
#Save the image
d = df.iloc[0]['src_subject_id']
t = df.iloc[0]['scan_session']
path=datapath+d+timepoint+t
first_img = nib.load(os.path.join(path,'smwc1pT1.nii')) # load the first image to get the header
stacked_nii_img = nib.Nifti1Image(stacked_image, first_img.affine, header=first_img.header)
nib.save(stacked_nii_img,'stacked_image.nii')
print(stacked_nii_img.shape)

In [ ]:
#Generating mean image and save it
mean_img = image.mean_img("stacked_image.nii")
print(mean_img)  
print("Shape:", mean_img.shape)  
print("Affine:\n", mean_img.affine)

nib.save(mean_img,'mean_image.nii')

In [ ]:
from scipy.stats import pearsonr
mean_img=image.mean_img("mean_image.nii")
mean_data = mean_img.get_fdata().flatten()

# Compute correlation for each image
images=[]
meta_data=[]
for _,s in df.iterrows():
    d=s['src_subject_id']
    t=s['scan_session']
    path=datapath+d+timepoint+t
    file_path=os.path.join(path,'smwc1pT1.nii')
    img=nib.load(file_path).get_fdata()
    img_data = img.flatten()  # Flatten current image
    corr, _ = pearsonr(img_data, mean_data)  # Compute Pearson correlation
    if corr>0.85:
        images.append(nib.load(file_path).get_fdata())
        meta_data.append(s)
print(len(images))
print(len(meta_data))

In [ ]:
corr_stacked_image = np.stack(images, axis=-1) 
print(corr_stacked_image.shape)
d = df.iloc[0]['src_subject_id']
t = df.iloc[0]['scan_session']
path=datapath+d+timepoint+t
first_img = nib.load(os.path.join(path,'smwc1pT1.nii')) # load the first image to get the header
corr_stacked_nii_img = nib.Nifti1Image(corr_stacked_image, first_img.affine, header=first_img.header)
nib.save(corr_stacked_nii_img,'corr_stacked_image.nii')
print(corr_stacked_nii_img.shape)

In [ ]:
smri_4d = image.load_img("corr_stacked_image.nii")
smri_data = smri_4d.get_fdata()
x_dim, y_dim, z_dim, subjects= smri_data.shape

print(smri_data.shape)

In [ ]:
average_map = np.mean(smri_data, axis=-1)
d = df.iloc[0]['src_subject_id']
t = df.iloc[0]['scan_session']
path=datapath+d+timepoint+t
first_img = nib.load(os.path.join(path,'smwc1pT1.nii')) # load the first image to get the header
average_img = nib.Nifti1Image(average_map,first_img.affine,header=first_img.header)
nib.save(average_img, 'average_map.nii')
plotting.view_img(average_img)

In [ ]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from nilearn import plotting

img = nib.load("average_map.nii")  
data = img.get_fdata()
plotting.view_img(img)

# Apply threshold to create a binary mask
threshold = 0.2
mask = (data > threshold).astype(np.uint8)  # 1 where data > 0.2, else 0

mask_nifti = nib.Nifti1Image(mask, img.affine, img.header)
nib.save(mask_nifti, "mask.nii")
plotting.view_img(mask_nifti)

# Visualize the middle slice
slice_idx = data.shape[-1] // 2  # Select a middle slice for visualization
plt.imshow(mask[:, :, slice_idx], cmap="gray")
plt.title("Binary Mask (Threshold > 0.2)")
plt.colorbar()
plt.show()


In [ ]:
Find the number of subjects for analysis

In [ ]:
df = pd.read_csv("Data.csv")  
df=df[df['imgincl_t1w_include']==1]

from scipy.stats import pearsonr
mean_img=image.mean_img("average_map.nii")
mean_data = mean_img.get_fdata().flatten()

# Compute correlation for each image
images=[]
meta_data=[]

for _,s in df.iterrows():
    d=s['src_subject_id']
    t=s['scan_session']
    path=datapath+d+timepoint+t
    file_path=os.path.join(path,'smwc1pT1.nii')
    img=nib.load(file_path).get_fdata()
    img_data = img.flatten()  # Flatten current image
    corr, _ = pearsonr(img_data, mean_data)  # Compute Pearson correlation
    if corr>0.85:
        images.append(nib.load(file_path).get_fdata())
        meta_data.append(s)
print(len(images))
print(len(meta_data))

filename='Corr_Data.csv'
meta_data_df=pd.DataFrame(meta_data)
meta_data_df.to_csv(filename)

Neurocombat for the subjects

In [ ]:
import os
import numpy as np
from nibabel.testing import data_path
import nibabel as nib
import matplotlib.pyplot as plt
import pandas as pd
from nilearn import image
datapath='/data/neuromark2/Data/ABCD/Data_BIDS_5/Raw_Data/'
timepoint="/Baseline/"
meta_data= pd.read_csv("Corr_Data.csv")  
images=[]
for _,s in meta_data.iterrows():
    d=s['src_subject_id']
    t=s['scan_session']
    path=datapath+d+timepoint+t
    file_path=os.path.join(path,'smwc1pT1.nii')
    images.append(nib.load(file_path).get_fdata()) 
stacked_image = np.stack(images, axis=-1) 

print(stacked_image.shape)

In [ ]:
path=datapath+d+timepoint+t
first_img = nib.load(os.path.join(path,'smwc1pT1.nii')) 
stacked_img = nib.Nifti1Image(stacked_image,first_img.affine,header=first_img.header)
nib.save(stacked_img, 'brain_images_abcd.nii')

In [ ]:
import numpy as np
import pandas as pd
import nibabel as nib
from neuroCombat import neuroCombat

nii_img = nib.load("brain_images_abcd.nii")  
images_4d = nii_img.get_fdata()  

X, Y, Z, num_subjects = images_4d.shape
num_voxels = X * Y * Z  

images_2d = images_4d.reshape(-1, num_subjects)  # Shape: (num_voxels, num_subjects)

meta_data_df = pd.read_csv("Corr_Data.csv") 
site_info = meta_data_df["site_id_l"].values 

assert len(site_info) == num_subjects, "Mismatch: site_info and number of subjects"

# Create DataFrame for neuroCombat
covars = pd.DataFrame({'site': site_info.astype(str)})  

# Convert data type to float32 for compatibility
images_2d = images_2d.astype(np.float32)

# Apply neuroCombat for harmonization
harmonized_data = neuroCombat(dat=images_2d, covars=covars, batch_col='site')

# Convert back to 4D NIfTI image
harmonized_4d = harmonized_data['data'].reshape(X, Y, Z, num_subjects)

# Save the harmonized image 
harmonized_img = nib.Nifti1Image(harmonized_4d, affine=nii_img.affine, header=nii_img.header)
nib.save(harmonized_img, "harmonized_brain_images_abcd.nii")
print("Shape: ",harmonized_img.shape)


In [ ]:
from nilearn.masking import apply_mask
import nibabel as nib
import numpy as np

mask_img=nib.load('mask.nii')
stacked_image=nib.load('harmonized_brain_images_abcd.nii')
print(stacked_image.shape)
masked_data = apply_mask(stacked_image, mask_img)

np.save("masked_data_abcd.npy", masked_data)
masked_data.shape